<a href="https://colab.research.google.com/github/Xuehui-Jiang/Uni-Mannheim-Master-Thesis/blob/main/Experimental_Evaluation/Entity_Relatedness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cloning the pyRDF2Vec repo
!git clone https://github.com/IBCNServices/pyRDF2Vec.git

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 7457, done.
remote: Counting objects: 100% (425/425), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 7457 (delta 268), reused 315 (delta 164), pack-reused 7032
Receiving objects: 100% (7457/7457), 5.42 MiB | 22.12 MiB/s, done.
Resolving deltas: 100% (5072/5072), done.


In [ ]:
!pip uninstall community
!pip install python-louvain
!pip install pyRDF2Vec --upgrade

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? Y
  Successfully uninstalled community-1.0.0b1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install rdflib
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import itertools
from typing import List, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdflib
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix

import sys
sys.path.append('pyRDF2Vec')
sys.path.append('pyRDF2Vec/pyrdf2vec')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving KORE.txt to KORE.txt
User uploaded file "KORE.txt" with length 19366 bytes


In [ ]:
# Read the KORE.txt file and clean up the entity URIs
entities = []

with open('KORE.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        cleaned_line = line.strip()  # Remove leading and trailing whitespaces
        if cleaned_line:
            entities.append(cleaned_line)

entities[:10]  # Show first 10 entities and the total number of entities


['http://dbpedia.org/resource/Apple_Inc.',
 'http://dbpedia.org/resource/Steve_Jobs',
 'http://dbpedia.org/resource/Steve_Wozniak',
 'http://dbpedia.org/resource/Jonathan_Ive',
 'http://dbpedia.org/resource/Mac_Pro',
 'http://dbpedia.org/resource/Mike_Markkula',
 'http://dbpedia.org/resource/Infinite_Loop_(street)',
 'http://dbpedia.org/resource/Silicon_Valley',
 'http://dbpedia.org/resource/NeXT',
 'http://dbpedia.org/resource/Safari_(web_browser)']

In [ ]:
print(len(entities))

441


In [ ]:
# 1. Read entities from KORE.txt and keep unique entities
unique_entities = set()

with open('KORE.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        cleaned_line = line.strip()  # Remove leading and trailing whitespaces
        if cleaned_line:
            unique_entities.add(cleaned_line)

unique_entities = list(unique_entities)  # Convert set to list

In [ ]:
print(len(unique_entities))

414


In [ ]:
# Filter out entities and relationships directly related to KORE Dataset
with open("/content/drive/My Drive/dbpedia_files/direct_merged_dbpedia.ttl", "r") as file, open("direct_KORE_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2:
            # Check if the subject or object of the triple is in our KORE URL list
            if triple[0].strip('<>') in entities or triple[2].strip('<>') in entities:
                out_file.write(line)

In [ ]:
# Filter out entities and relationships directly or indirectly related to KORE Dataset
entities_related_to_KORE = set(entities)
new_entities = set(entities)
iteration_count = 0

while new_entities and iteration_count < 2:
    temp_entities = set()
    with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file:
        for line in file:
            triple = line.strip().split()
            if len(triple) > 2 and (triple[0].strip('<>') in new_entities or triple[2].strip('<>') in new_entities):
                    temp_entities.add(triple[0].strip('<>'))
                    temp_entities.add(triple[2].strip('<>'))

    new_entities = temp_entities - entities_related_to_KORE
    entities_related_to_KORE.update(new_entities)
    iteration_count += 1

# Finally, we can write all triples that are directly or indirectly related to entities in the `entities_related_to_KORE` collection
with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file, open("indirect_KORE_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2 and (triple[0].strip('<>') in entities_related_to_KORE or triple[2].strip('<>') in entities_related_to_KORE):
            out_file.write(line)


In [ ]:
# Define the list of files
files_to_merge = [
    "direct_KORE_filtered_dbpedia.ttl",
    "indirect_KORE_filtered_dbpedia.ttl"
]

# Merge files
with open("KORE_filtered_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open(file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
input_file = "KORE_filtered_dbpedia.ttl"
output_file = "cleaned_KORE_filtered_dbpedia.ttl"

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Determine whether it is a comment or a blank line
        if line.startswith("#") or line.strip() == "":
            continue

        # Split triples
        triple = line.strip().split()

        # Make sure this is a complete triple
        if len(triple) > 2:
            # Check whether the object starts with "http://dbpedia.org/resource/"
            if triple[2].startswith("<http://dbpedia.org/resource/"):
                outfile.write(line)

print(f"Finished filtering {input_file} and saved the result to {output_file}")


Finished filtering KORE_filtered_dbpedia.ttl and saved the result to cleaned_KORE_filtered_dbpedia.ttl


In [ ]:
!head -n 10 cleaned_KORE_filtered_dbpedia.ttl

<http://dbpedia.org/resource/Aikido> <http://dbpedia.org/property/country> <http://dbpedia.org/resource/Japan> .
<http://dbpedia.org/resource/American_Film_Institute> <http://dbpedia.org/property/keyPeople> <http://dbpedia.org/resource/Bob_Gazzale> .
<http://dbpedia.org/resource/American_Film_Institute> <http://dbpedia.org/property/keyPeople> <http://dbpedia.org/resource/Howard_Stringer> .
<http://dbpedia.org/resource/American_Film_Institute> <http://dbpedia.org/property/keyPeople> <http://dbpedia.org/resource/Robert_A._Daly> .
<http://dbpedia.org/resource/Apple_Inc.> <http://dbpedia.org/property/tradedAs> <http://dbpedia.org/resource/Dow_Jones_Industrial_Average> .
<http://dbpedia.org/resource/Apple_Inc.> <http://dbpedia.org/property/tradedAs> <http://dbpedia.org/resource/NASDAQ-100> .
<http://dbpedia.org/resource/Apple_Inc.> <http://dbpedia.org/property/tradedAs> <http://dbpedia.org/resource/S&P_500> .
<http://dbpedia.org/resource/Apple_Inc.> <http://dbpedia.org/property/founders> <h

In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("cleaned_KORE_filtered_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("/content/drive/My Drive/dbpedia_files/cleaned_KORE_filtered_dbpedia.ttl", fmt='turtle')

kgentities = kg._entities

In [ ]:
for e in unique_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/Tomb_Raider_(video_game)
http://dbpedia.org/resource/Templeton_"Faceman"_Peck


In [ ]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/Tomb_Raider_(video_game)",
    "http://dbpedia.org/resource/Templeton_\"Faceman\"_Peck"
]

# Filter out absent entities
filtered_entities = [e for e in unique_entities if e not in absent_entities]

In [ ]:
print(len(filtered_entities))

412


In [ ]:
# Read the file and split it by lines
with open('KORE.txt', 'r') as file:
    lines = file.readlines()

# Initialize the main_entities and related_entities
main_entities = []
related_entities = {}

# Parse each line based on the structure explained
for line in lines:
    if line.startswith('\t'):
        # It's a related entity (starts with tab), remove the tab and append it to the current main entity's list
        related_entities[current_main_entity].append(line[1:].strip())
    else:
        # It's a main entity (does not start with tab), also strip to remove any leading/trailing whitespaces
        current_main_entity = line.strip()
        main_entities.append(current_main_entity)
        related_entities[current_main_entity] = []

# Create a dict: main entities as keys and their corresponding related entities as values
groups = {key: related_entities[key] for key in main_entities}


In [ ]:
# Create DataFrame list - right_ignored_list
missing_entities = [
    "http://dbpedia.org/resource/Tomb_Raider_(video_game)",
    "http://dbpedia.org/resource/Templeton_\"Faceman\"_Peck"
]

# Initialize the list that will hold the dataframes for each main entity
right_ignored_list = []

# Loop through each main entity to create its ignored dataframe
for main_entity in main_entities:
    ignored_for_this_main = []

    # Check if each missing entity is in the related entities of the current main entity
    for missing_entity in missing_entities:
        if missing_entity in related_entities[main_entity]:
            ignored_for_this_main.append(missing_entity)

    # Create a DataFrame for the ignored entities for this main entity
    df_ignored = pd.DataFrame(ignored_for_this_main, columns=['name'])
    right_ignored_list.append(df_ignored)

print(len(right_ignored_list))

21


In [ ]:
# Create left_merged/right_merged DataFrame dynamically
def create_dataframe_from_embeddings(entity_list, embedding_dict):
    # Only include entities that have embeddings
    valid_entities = [e for e in entity_list if e in embedding_dict]

    data = {'name': valid_entities}
    for entity in valid_entities:
        if entity in embedding_dict:
            embedding_vector = embedding_dict[entity]
            for dim_idx, dim_value in enumerate(embedding_vector):
                dim_name = f'v{dim_idx + 1}'
                if dim_name not in data:
                    data[dim_name] = []
                data[dim_name].append(dim_value)
        else:
            print(f"Warning: No embedding found for entity {entity}")

    return pd.DataFrame(data)

In [ ]:
from scipy.spatial import distance
from scipy.stats import kendalltau
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def train(left_merged, left_ignored, right_merged_list, right_ignored_list, groups):
    """
    It trains the model based on the provided data

    left_merged: dataframe containing main entities and the related vectors
    left_ignored: dataframe containing the missing main entities, i.e. the entities which are in the dataset but not in the input file
    right_merged_list: list of dataframes containing the right entities and the related vectors
    right_ignored_list: list of dataframes containing the missing right entities, i.e. the entities which are in the dataset but not in the input file
    groups: dictionary containing the main entities and the attached right entities

    It returns the result object reporting the task name, the model name and its configuration - if any -, and the evaluation metric values.
    """

    predicted_rank_list = compute_relatedness(
        left_merged, left_ignored, right_merged_list, right_ignored_list)
    gold_rank_list = np.tile(np.arange(1, 21), (21, 1))
    return evaluate_ranking(list(groups.keys()), gold_rank_list, predicted_rank_list)

In [ ]:
def compute_relatedness(left_merged, left_ignored, right_merged_list, right_ignored_list):
    """
    It computes the relatedness among main and right entities.

    left_merged: dataframe containing main entities and the related vectors
    left_ignored: dataframe containing the missing main entities, i.e. the entities which are in the dataset but not in the input file
    right_merged_list: list of dataframes containing the right entities and the related vectors
    right_ignored_list: list of dataframes containing the missing right entities, i.e. the entities which are in the dataset but not in the input file

    It returns the predicted ranked list.
    """
    predicted_rank_list = list()

    for i in range(len(left_merged["name"])):
        distances = []

        if right_merged_list[i].size > 0:
            # Use sklearn's cosine_similarity function to calculate cosine similarity
            similarities = cosine_similarity(
                left_merged.iloc[[i], 1:],
                right_merged_list[i].iloc[:, 1:]
            )
            # Convert similarity to distance: distance = 1 - similarity
            distances = 1 - similarities.flatten()

        ignored_distances = np.array(
            [1 for j in range(len(right_ignored_list[i]))]
        )  # max dist??
        distances = np.concatenate((distances, ignored_distances))

        predicted_rank_list.append(list(np.argsort(distances)))

    for i in range(len(left_ignored["name"])):
        predicted_rank_list.append(list(np.argsort([1 for i in range(20)])))

    return predicted_rank_list


In [ ]:
def compute_relatedness(left_merged, left_ignored, right_merged_list, right_ignored_list):
    """
    It computes the relatedness among main and right entities.

    left_merged: dataframe containing main entities and the related vectors
    left_ignored: dataframe containing the missing main entities, i.e. the entities which are in the dataset but not in the input file
    right_merged_list: list of dataframes containing the right entities and the related vectors
    right_ignored_list: list of dataframes containing the missing right entities, i.e. the entities which are in the dataset but not in the input file

    It returns the predicted ranked list.
    """
    predicted_rank_list = list()

    for i in range(len(left_merged["name"])):
        distances = []

        if right_merged_list[i].size > 0:
            distances = distance.cdist(
                left_merged.iloc[[i], 1:],
                right_merged_list[i].iloc[:, 1:],
                metric=distance.euclidean,
            ).flatten()

        ignored_distances = np.array(
            [1 for j in range(len(right_ignored_list[i]))]
        )  # max dist??
        distances = np.concatenate((distances, ignored_distances))

        predicted_rank_list.append(list(np.argsort(distances)))

    for i in range(len(left_ignored["name"])):
        predicted_rank_list.append(list(np.argsort([1 for i in range(20)])))

    return predicted_rank_list


In [ ]:
def evaluate_ranking(entities_list, gold_ranking_list, predicted_ranking_list, debugging_mode=True):
    """
    It evaluates the ranking comparing the predicted and the gold one.

    entities_list: it keeps the relation among the main entities and the right entities
    gold_ranking_list: list of the ranking used as gold standard
    predicted_ranking_list: list of the predicted ranking
    debugging_mode: {TRUE, FALSE}, TRUE to run the model by reporting all the errors and information; FALSE otherwise

    It returns the evaluation of the predicted ranking.
    """

    score_list = list()
    float_precision = 4

    for i in range(len(entities_list)):
        if debugging_mode:
            print("Entity Relatedness : " + entities_list[i])
            print(gold_ranking_list[i])
            print(predicted_ranking_list[i])
        kendalltau_correlation, kendalltau_pvalue = kendalltau(
            gold_ranking_list[i], predicted_ranking_list[i]
        )
        if debugging_mode:
            print(
                "Entity Relatedness : "
                + entities_list[i]
                + " kendalltau correlation "
                + str(kendalltau_correlation)
                + " kendall tau pvalue "
                + str(kendalltau_pvalue)
            )
        score_list.append(
            {
                "task_name": "EntityRelatedness",
                "entity_name": entities_list[i],
                "kendalltau_correlation": round(kendalltau_correlation, float_precision),
                "kendalltau_pvalue": round(kendalltau_pvalue, float_precision),
            }
        )

    return score_list

In [ ]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # 2. Create DataFrames for Entity Relatedness
        left_merged = create_dataframe_from_embeddings(main_entities, embedding_dict)
        right_merged_list = [create_dataframe_from_embeddings(rel, embedding_dict) for rel in related_entities.values()]
        left_ignored = pd.DataFrame(columns=['name'])

        # 3. Call the Entity Relatedness evaluation method
        scores = train(left_merged, left_ignored, right_merged_list, right_ignored_list, groups)

        # 4. Collect and save results
        avg_score = np.mean([s['kendalltau_correlation'] for s in scores])
        results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Averge Score': avg_score})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('entity_relatedness_scores.csv', index=False)


Entity Relatedness : http://dbpedia.org/resource/Apple_Inc.
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[11, 7, 3, 9, 0, 17, 8, 4, 10, 18, 5, 19, 1, 2, 12, 13, 14, 6, 15, 16]
Entity Relatedness : http://dbpedia.org/resource/Apple_Inc. kendalltau correlation 0.23157894736842105 kendall tau pvalue 0.16497640636360544
Entity Relatedness : http://dbpedia.org/resource/Google
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[11, 7, 6, 2, 1, 5, 3, 10, 8, 4, 15, 14, 9, 13, 0, 12, 18, 16, 19, 17]
Entity Relatedness : http://dbpedia.org/resource/Google kendalltau correlation 0.4421052631578947 kendall tau pvalue 0.005923797271372209
Entity Relatedness : http://dbpedia.org/resource/Facebook
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[5, 4, 15, 19, 11, 9, 16, 1, 2, 12, 8, 10, 7, 18, 13, 0, 3, 14, 6, 17]
Entity Relatedness : http://dbpedia.org/resource/Facebook kendalltau correlation 0.010526315789473684 kendall tau pvalue 0.9744669029388741
Entity 

In [ ]:
print(results_df)

  Embedding Mode  Dimensions  Averge Score
0           cbow          50      0.182457
1           cbow         100      0.215033
2           cbow         200      0.203500
3             sg          50      0.291233
4             sg         100      0.299248
5             sg         200      0.293733


Prepare for Knowledge Graph

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
import networkx as nx

In [ ]:
# Load the graph from the input file
graph = Graph()
graph.parse("/content/drive/My Drive/dbpedia_files/cleaned_KORE_filtered_dbpedia.ttl", format="turtle")

<Graph identifier=Nb28cd33e583c4bb8bb772af000e67f63 (<class 'rdflib.graph.Graph'>)>

In [ ]:
G = nx.DiGraph()  # Create a directed graph

for s, p, o in graph:
    G.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G.nodes())
num_edges = len(G.edges())
print("The DBpedia knowledge graph as of 2016-10 has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The DBpedia knowledge graph as of 2016-10 has 4127548 nodes and 11963841 edges.


graph embeddings with SA in BFS

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving sa_helper.py to sa_helper.py


In [ ]:
from main import *
from sa_helper import *

In [ ]:
# Count the neighbor node number for each nodes in the graph
neighbor_counts = {node: len(list(G.neighbors(node))) for node in G.nodes()}

In [ ]:
subgraph_F_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub.nodes())
num_edges = len(G_sub.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 184331 nodes and 203229 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # 2. Create DataFrames for Entity Relatedness
        left_merged = create_dataframe_from_embeddings(main_entities, embedding_dict)
        right_merged_list = [create_dataframe_from_embeddings(rel, embedding_dict) for rel in related_entities.values()]
        left_ignored = pd.DataFrame(columns=['name'])

        # 3. Call the Entity Relatedness evaluation method
        scores = train(left_merged, left_ignored, right_merged_list, right_ignored_list, groups)

        # 4. Collect and save results
        avg_score = np.mean([s['kendalltau_correlation'] for s in scores])
        results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Averge Score': avg_score})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('entity_relatedness_scores_BFS.csv', index=False)


Entity Relatedness : http://dbpedia.org/resource/Apple_Inc.
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[2, 9, 19, 5, 3, 7, 16, 17, 18, 11, 10, 12, 1, 14, 6, 0, 8, 13, 4, 15]
Entity Relatedness : http://dbpedia.org/resource/Apple_Inc. kendalltau correlation -0.010526315789473684 kendall tau pvalue 0.9744669029388741
Entity Relatedness : http://dbpedia.org/resource/Google
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[12, 13, 0, 7, 14, 17, 15, 1, 10, 5, 11, 16, 18, 2, 4, 9, 19, 6, 3, 8]
Entity Relatedness : http://dbpedia.org/resource/Google kendalltau correlation -0.031578947368421054 kendall tau pvalue 0.8728414598639421
Entity Relatedness : http://dbpedia.org/resource/Facebook
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[9, 4, 2, 1, 11, 8, 5, 14, 16, 10, 13, 7, 15, 19, 0, 12, 6, 18, 3, 17]
Entity Relatedness : http://dbpedia.org/resource/Facebook kendalltau correlation 0.24210526315789474 kendall tau pvalue 0.1458442183647273
Entity

with DFS

In [ ]:
subgraph_F_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub_ff = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub_ff.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ff.nodes())
num_edges = len(G_sub_ff.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 184330 nodes and 203229 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # 2. Create DataFrames for Entity Relatedness
        left_merged = create_dataframe_from_embeddings(main_entities, embedding_dict)
        right_merged_list = [create_dataframe_from_embeddings(rel, embedding_dict) for rel in related_entities.values()]
        left_ignored = pd.DataFrame(columns=['name'])

        # 3. Call the Entity Relatedness evaluation method
        scores = train(left_merged, left_ignored, right_merged_list, right_ignored_list, groups)

        # 4. Collect and save results
        avg_score = np.mean([s['kendalltau_correlation'] for s in scores])
        results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Averge Score': avg_score})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('entity_relatedness_scores_DFS.csv', index=False)


Entity Relatedness : http://dbpedia.org/resource/Apple_Inc.
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[9, 2, 19, 5, 1, 18, 11, 7, 17, 16, 3, 10, 12, 14, 8, 6, 0, 13, 4, 15]
Entity Relatedness : http://dbpedia.org/resource/Apple_Inc. kendalltau correlation -0.042105263157894736 kendall tau pvalue 0.8226828854421667
Entity Relatedness : http://dbpedia.org/resource/Google
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[12, 13, 1, 0, 7, 17, 15, 14, 10, 9, 11, 5, 2, 16, 4, 18, 6, 3, 19, 8]
Entity Relatedness : http://dbpedia.org/resource/Google kendalltau correlation 0.021052631578947368 kendall tau pvalue 0.9235023313316655
Entity Relatedness : http://dbpedia.org/resource/Facebook
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[4, 9, 2, 1, 5, 8, 11, 14, 16, 13, 0, 7, 19, 10, 15, 12, 6, 18, 17, 3]
Entity Relatedness : http://dbpedia.org/resource/Facebook kendalltau correlation 0.2947368421052632 kendall tau pvalue 0.07402572925713644
Entity 